## Practica 3 (Parte 2)
#### Alberto García Doménech - Pablo Daurell Marina

**Ti** = trabajador i   
**tj** = tarea j   
**dij** = Tiempo que tarda Ti en realizar tj   

|        |  t1  |  t2  |  t3  |
|--------|------|------|------|
| **T1** | d11  | d12  | d13  |
| **T2** | d21  | d22  | d23  |

- Cada fila representa un trabajador
- Cada columna representa una tarea
    - Suponemos que las tareas aparecen ya ordenadas siguiendo una restriccón temporal (t1 debe realizarse antes que t2)
    
    
- Resolveremos este problema usando un algoritmo genético (basándonos en el problema de la mochila)
- La solución consistirá de una matriz, como la mostrada arriba, con valores binarios 
    > Si la casilla dij es 1 => el trabajador i realiza la tarea j   
    > Si la casilla dij es 0 => El trabajador i no realiza la tarea j

In [93]:
import random

Definimos la clase para nuestro problema genético pasando como parámetros:
- Los genes utilizados (0s y 1s)
- La función de decodificado
- La función de mutación
- La función de cruzado
- La función de fitness
- El número de tareas (len_col)
- El número de trabajadores (len_row)

In [94]:
class ProblemaGenetico(object):
        def __init__(self, genes,fun_dec,fun_muta , fun_cruza, fun_fitness, len_col, len_row):
            self.genes = genes
            self.fun_dec = fun_dec
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.len_row = len_row
            self.len_col = len_col
            """Constructor de la clase"""
                
        def decodifica(self, genotipo):
            """Devuelve el fenotipo a partir del genotipo"""
            fenotipo = self.fun_dec(genotipo, self.len_col, self.len_row)
            return fenotipo
        def muta(self, cromosoma,prob):
            """Devuelve el cromosoma mutado"""   
            mutante = self.fun_muta(cromosoma,prob,self.len_col,self.len_row)
            return mutante
        def cruza(self, cromosoma1, cromosoma2):         
            """Devuelve el cruce de un par de cromosomas"""
            cruce = self.fun_cruza(cromosoma1,cromosoma2, self.len_col, self.len_row)
            return cruce 
        def fitness(self, cromosoma):    
            """Función de valoración"""
            valoracion = self.fun_fitness(cromosoma, self.len_col, self.len_row)
            return valoracion

In [95]:
# Ej ({T1,T2}, {t1, t2, t3})
# costs = [[1,3,2],
#          [2,1,2]]

inicial = [[1,1,0],
           [1,0,1]]
tiempos = [[3,5,1],
           [2,8,1]]
n_tareas = 3
n_trabajadores = 2

### Decodificación
La funcion de decodificado recorre un cromosoma y lo convierte en una solución valida para nuestro problema.   
Tiene en cuenta que haya exactamente un 1 en cada columna (una tarea solo puede hacerla una persona) y si no hay ninguno añade un en una posición aleatoria de la columna.

In [96]:
def deco(cromosoma, n_tareas, n_trabajadores):
    l = cromosoma
    for i in range(n_tareas):
        for j in range(n_trabajadores):
            if cromosoma[j][i] == 1:
                for k in range(j+1, n_trabajadores):
                    l[k][i] = 0
                break
            elif j == n_trabajadores - 1:
                l[random.randrange(0, n_trabajadores,1)][i] = 1
    return l

In [97]:
deco(inicial, n_tareas, n_trabajadores)

[[1, 1, 0], [0, 0, 1]]

### Fitness
La función de fitness recibe los tiempos que tarda cada trabajador en hacer cada tarea.  
Va acumulando el tiempo total que se tardaría en completar nuestra solución, sumando el tiempo que tarda cada tarea en ser completada por el trabajador asignado.

In [98]:
def fitness(cromosoma, n_tareas, n_trabajadores, tiempos):
    tiempo_total = 0
    cromosoma = deco(cromosoma, n_tareas, n_trabajadores)
    for i in range(n_tareas):
        for j in range(n_trabajadores):
            if cromosoma[j][i] == 1:
                tiempo_total += tiempos[j][i]
    return tiempo_total

In [99]:
fitness(inicial, n_tareas, n_trabajadores, tiempos)

9

### Cruzado y mutado
La función de cruce usa cruce de un punto (junta la primera mitad del cromosoma1 con la segunda mitad del cromosoma2 y viceversa).   
La función de mutación elige un gen al azar del cromosoma y si la probabilidad es mayor que un número al azar altera el valor de ese gen.

In [100]:
def fun_cruzar(cromosoma1, cromosoma2, n_tareas, n_trabajadores):
    """Cruza los cromosomas por la mitad"""
    cruce1 = cromosoma1[0:n_trabajadores//2][0:n_tareas]+cromosoma2[n_trabajadores//2:n_trabajadores][0:n_tareas]
    cruce2 = cromosoma2[0:n_trabajadores//2][0:n_tareas]+cromosoma1[n_trabajadores//2:n_trabajadores][0:n_tareas]
    return [cruce1,cruce2]
    
def fun_mutar(cromosoma,prob, n_tareas, n_trabajadores):
    """Elige un elemento al azar del cromosoma y lo modifica con una probabilidad igual a prob"""
    p = random.randint(0, n_trabajadores-1)
    q = random.randint(0,n_tareas-1)
    if prob > random.uniform(0,1):
        cromosoma[p][q] =  (cromosoma[p][q]+1)%2
    return cromosoma

problema = ProblemaGenetico([0,1], deco, fun_mutar, fun_cruzar, fitness, n_tareas,n_trabajadores)

### Población inicial
Generamos aleatoriamente una lista de matrices (que representan a cada individuo) representando así una población inicial.

In [101]:
def poblacion_inicial(problema_genetico, size):
    l = []
    for k in range(size):
        row = []
        for i in range(problema_genetico.len_row):
            col = []
            for j in range(problema_genetico.len_col):
                col.append(random.choice(problema_genetico.genes))
            row.append(col)
        l.append(row)
    return l

- Creamos funciones para cruzar y mutar todos los individuos de una población

In [102]:
def cruza_padres(problema_genetico, padres):
    l = []
    for i in range(len(padres)-1):
        x = problema_genetico.cruza(padres[i], padres[i+1])
        l.append(x[0])
        l.append(x[1])
    return l

In [103]:
def muta_individuos(problema_genetico, poblacion, prob):
    l = []
    for x in poblacion:
        l.append(problema_genetico.muta(x, prob))
    return l

Utilizamos las funciones de selección por torneo, nueva generación y el algoritmo genético creados en la parte uno de la práctica (para resolver el problema de la mochila) sin modificarlos ya que sirven para nuestra implementación.

In [104]:
def seleccion_por_torneo(problema_genetico, poblacion, n, k, opt):
    """Selección por torneo de n individuos de una población. Siendo k el nº de participantes
        y opt la función max o min."""
    seleccionados = []
    for i in range(n):
        participantes = random.sample(poblacion,k)
        seleccionado = opt(participantes, key=problema_genetico.fitness)
        opt(poblacion, key=problema_genetico.fitness)
        seleccionados.append(seleccionado)
        # poblacion.remove(seleccionado)
    return seleccionados  

In [105]:
def algoritmo_genetico(problema_genetico,k,opt,ngen,size,prop_cruces,prob_mutar):
    poblacion= poblacion_inicial(problema_genetico,size)
    n_padres=round(size*prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = size-n_padres
    for _ in range(ngen):
        poblacion= nueva_generacion(problema_genetico,k,opt,poblacion,n_padres, n_directos,prob_mutar)

    mejor_cr= opt(poblacion, key=problema_genetico.fitness)
    mejor=problema_genetico.decodifica(mejor_cr)
    return (mejor,problema_genetico.fitness(mejor_cr)) 

In [106]:
def nueva_generacion(problema_genetico, k,opt, poblacion, n_padres, n_directos, prob_mutar):
    padres2 = seleccion_por_torneo(problema_genetico, poblacion, n_directos, k,opt) 
    padres1 = seleccion_por_torneo(problema_genetico, poblacion, n_padres , k, opt)
    cruces =  cruza_padres(problema_genetico,padres1)
    generacion = padres2+cruces
    resultado_mutaciones = muta_individuos(problema_genetico, generacion, prob_mutar)
    return resultado_mutaciones

### Resolución
Creamos un ejemplo de nuestro problema, definimos la función de fitness concreta para este caso y resolvemos el problema mediante el algoritmo genético.

In [107]:
inicial = [[1,1,0],
           [1,0,1]]
tiempos = [[3,5,1],
           [2,8,1]]
n_tareas = 3
n_trabajadores = 2

def fitness_tareas_1(cromosoma, n_tareas, n_trabajadores):
    v = fitness(cromosoma, n_tareas, n_trabajadores, tiempos)
    return v

problema1 = ProblemaGenetico([0,1],deco,fun_mutar,fun_cruzar,fitness_tareas_1,n_tareas,n_trabajadores)

In [114]:
algoritmo_genetico(problema1,5,min,100,10,0.8,0.05)

([[0, 1, 1], [1, 0, 0]], 8)

In [115]:
algoritmo_genetico(problema1,5,min,100,10,0.8,0.05)

([[1, 1, 1], [0, 0, 0]], 9)

Vemos que se siempre obtiene una solución válida, pero no siempre podemos garantizar que se consiga la más óptima (8).   
Esto se debe a que en la búsqueda local podemos quedarnos atascados en óptimos locales.